In [1]:
import pathlib
import re
import collections
import functools
import difflib
import operator
import networkx as nx
import warnings
import pickle
warnings.simplefilter('ignore') # for networkx drawing
import matplotlib.pylab
%matplotlib inline
matplotlib.pylab.rcParams["figure.figsize"] = (14, 18)

In [2]:
datadir = pathlib.Path("/home/david/pro/scc/data")

In [3]:
%run utils.py
#cache = ApprovedMatches(datadir / "approved.pickle")

# Fix the birthplace field

In [19]:
birthplaces = collections.Counter()
changes = set()
for fn in sorted(datadir.glob("lc_*.csv"))[0:1]:
    year = int(re.search(r"\d{4}", fn.name).group(0))
    print(year)
    with fn.open("r", encoding="UTF-8") as fd:
        next(fd)
        for line in fd:
            row = line.strip().split("|")
            # prøv at fikse fødesteder
            birthplace = row[5]
            if "her i sognet" in birthplace or "heri sognet" in birthplace or birthplace.replace(".", "") in ("i sognet", "her i s"):        
                changes.add((birthplace, row[2]))
                birthplace = row[2]
            birthplaces.update((birthplace,))
            continue
            fødested = row[5].replace(".","").split()
            if len(fødested) > 1:
                if fødested[-2:] == ["i", "sognet"] or fødested == ["heri", "sognet"] or "her i s" in row[5]: # {her ,}i sognet
                    fødested = row[2] # erstat med sognet
                elif "sogn" in fødested: # ... sogn på læsø / ... sogn ... amt
                    index = fødested.index("sogn")
                    fødested = " ".join(fødested[:index])
                elif "amt" in fødested:
                    index = fødested.index("amt") # ... by amt
                    fødested = " ".join(fødested[:index-1])
                elif fødested[-1].startswith("["): # ... [by]
                    fødested = " ".join(fødested[:-1])
                else:
                    fødested = " ".join(fødested)
                    unhandled.add(fødested)
                row[5] = fødested
            row = Entry(*row)

1845


In [21]:
sorted(changes)

[('<heri sognet', 'vindblæs'),
 ('>ditto=[her i sognet]', 'vinderød'),
 ('alle 5 børn var født heri sognet', 'budolfi landsogn'),
 ('alle er fødte her i sognet', 'borre'),
 ('alle fødte her i sognet', 'borre'),
 ('alle fødte her i sognet', 'farup (kongerigsk)'),
 ('alle fødte her i sognet', 'torslunde'),
 ('alle fødte heri sognet', 'torslunde'),
 ('alle her i sognet', 'torslunde'),
 ('baulstrup her i sognet', 'sønderup'),
 ('benzondahl, her i sognet', 'torslunde'),
 ('bistrup, her i sognet', 'fredensborg slot'),
 ('braedstrup her i sognet', 'sønderup'),
 ('braulstrup bye her i sognet', 'sønderup'),
 ('braulstrup her i sognet', 'sønderup'),
 ('brohus, her i sognet [Østerhæsinge]', 'Øster hæsinge'),
 ('brohuset, her i sognet [Østerhæsinge]', 'Øster hæsinge'),
 ('brøndum by heri sognet', 'skarp salling'),
 ('brøndum heri sognet', 'skarp salling'),
 ('brøndum heri sognet aalborg amt', 'skarp salling'),
 ('d [her i sognet]', 'sakskøbing landdistrikt'),
 ('d. [her i sognet]', 'radsted'),
 ('

In [18]:
birthplaces.most_common()

[('kjøbenhavn', 46589),
 ('københavn', 20920),
 ('odense', 8211),
 ('kiøbenhavn', 7701),
 ('aalborg', 5928),
 ('randers', 5727),
 ('horsens', 4174),
 ('aarhus', 4048),
 ('helsingør.', 3709),
 ('fredericia', 3592),
 ('her i sognet [tikøb]', 3573),
 ('rønne købstad - bornholms amt', 2996),
 ('slagelse', 2813),
 ('taarnby sogn, københavns amt', 2804),
 ('dronninglund', 2634),
 ('født i sognet', 2468),
 ('sognet', 2415),
 ('do [heri sognet]', 2408),
 ('assens', 2142),
 ('her i sognet, kalvehave', 2095),
 ('svendborg', 2065),
 ('nyborg', 2046),
 ('klemensker sogn - bornholms amt', 2043),
 ('Åker sogn - bornholms amt', 2026),
 ('højby', 2002),
 ('roeskilde', 1888),
 ('ribe', 1875),
 ('nakskov', 1852),
 ('vejle', 1828),
 ('draaby', 1828),
 ('toreby', 1799),
 ('Østermarie sogn - bornholms amt', 1779),
 ('do [horne]', 1777),
 ('birkerød', 1655),
 ('aarhuus', 1642),
 ('her i sognet [nordby sogn][ribe amt]', 1621),
 ('her', 1593),
 ('vestermarie sogn - bornholms amt', 1548),
 ('karlebo', 1539),
 